In [1]:
from pyspark.sql import SparkSession
import findspark

findspark.init()
spark = SparkSession.builder.appName('challenge').getOrCreate()

In [2]:
spark.read.parquet('./optimized_data/data_1/').createOrReplaceTempView('data_1')
spark.read.parquet('./optimized_data/data_2/').createOrReplaceTempView('data_2')
spark.read.parquet('./optimized_data/data_3/').createOrReplaceTempView('data_3')

This logic is used to merge every dataset according the analysis intention. A similar approach was used on the other analysis.

In [3]:
#   - **Compensation by Role**
#     - Find the average compensation of roles where the role is some kind of technical professional
#     - Visualize the relationship between compensation and role
job_roles_and_compensation_exprs = [f'''
    SELECT job_title, annual_salary 
    FROM data_{i} 
    WHERE job_title IS NOT NULL
    AND annual_salary IS NOT NULL
''' for i in range(1, 4)]

spark.sql(' UNION '.join(job_roles_and_compensation_exprs)).createOrReplaceTempView('roles_and_compensation')

Trying to standardize the `job_title` for a better aggregation, every value was converted to TitleCase and **Junior** and **Senior** became **Jr.** and **Sr.** respectively. This does not prevents, for example, of Senior being a prefix or a suffix of `job_title`, which should be treated as the same. Also, there are invalid values, like *Test*, that should be ignored on the final analysis.

In [4]:
spark.sql('''
    SELECT replace(replace(initcap(job_title), 'Junior', 'Jr.'), 'Senior', 'Sr.'), count(1), bround(AVG(annual_salary), 2)
    FROM roles_and_compensation
    GROUP BY 1
    ORDER BY 3 DESC
''').show(10000, truncate=False)

+----------------------------------------------------------------------------------+--------+-----------------------------+
|replace(replace(initcap(job_title), Junior, Jr.), Senior, Sr.)                    |count(1)|bround(avg(annual_salary), 2)|
+----------------------------------------------------------------------------------+--------+-----------------------------+
|Ggggggggggggg                                                                     |1       |1111111112221.00             |
|Qwdqwd                                                                            |1       |123123123123.00              |
|꼰대                                                                              |1       |9999999999.00                |
|Abap                                                                              |1       |200000000.00                 |
|R&d Intern                                                                        |1       |156000000.00                 |
|Professsi

To remove most invalid `job_titles`, the logic of removing every one that appeared only once was used. Unfortunately, this can also remove valid jobs from the dataset, and does not fully prevents invalid jobs (like *Test*)

In [5]:
spark.sql('''
    SELECT replace(replace(initcap(job_title), 'Junior', 'Jr.'), 'Senior', 'Sr.') AS job_title, 
           count(1) AS count, 
           bround(AVG(annual_salary), 2) AS avg_salary
    FROM roles_and_compensation
    GROUP BY 1
    HAVING count(1) > 1
    ORDER BY 3 DESC
''').show(10000, truncate=False)

+----------------------------------------------+-----+-----------+
|job_title                                     |count|avg_salary |
+----------------------------------------------+-----+-----------+
|Test                                          |6    |12542263.17|
|Design Manager                                |5    |9686126.00 |
|Products Engineer                             |2    |7920000.00 |
|Hr Generalist                                 |34   |3939262.12 |
|Associate Attorney                            |58   |3723327.59 |
|Worker                                        |3    |3383333.33 |
|Legal Editor                                  |2    |3085500.00 |
|Writer                                        |17   |2234058.82 |
|Line Producer                                 |3    |1896666.67 |
|Curriculum Coordinator                        |2    |1842500.00 |
|Cto                                           |46   |1701197.83 |
|Data Science                                  |4    |1568250.

In [6]:
#   - **Compensation by Geographic Region**
#     - Average, min, and max compensation per city/region
#     - Visualize the relationship between city and salary/base compensation
region_and_compensation_exprs = [f'''
    SELECT location, annual_salary 
    FROM data_{i} 
    WHERE location IS NOT NULL
    AND annual_salary IS NOT NULL
''' for i in [1, 3]
]

region_and_compensation_exprs.append('''
    SELECT country || city AS location, annual_salary 
    FROM data_2 
    WHERE country IS NOT NULL
    AND city IS NOT NULL
    AND annual_salary IS NOT NULL
''')

spark.sql(' UNION '.join(region_and_compensation_exprs)).createOrReplaceTempView('region_and_compensation')

In [7]:
spark.sql('''
    SELECT regexp_replace(initcap(location), '([^a-zA-Z ])', '') AS location, 
           count(1) AS count,
           bround(avg((annual_salary)), 2) AS avg_salary, 
           bround(min((annual_salary)), 2) AS min_salary,
           bround(max((annual_salary)), 2) AS max_salary
    FROM region_and_compensation
    GROUP BY 1
    ORDER BY 2
''').show(truncate=False)

+---------------------------------------+-----+----------+----------+----------+
|location                               |count|avg_salary|min_salary|max_salary|
+---------------------------------------+-----+----------+----------+----------+
|Danville Pa Usa                        |1    |48.00     |48.00     |48.00     |
|Nigga                                  |1    |8675309.00|8675309.00|8675309.00|
|Park City Utah  Usa                    |1    |54000.00  |54000.00  |54000.00  |
|Pompton Lakesnjusa                     |1    |32000.00  |32000.00  |32000.00  |
|Western Suburbs Of Chicago Illinois Usa|1    |70000.00  |70000.00  |70000.00  |
|West Midlands Uk                       |1    |45000.00  |45000.00  |45000.00  |
|West Vancouver                         |1    |74345.00  |74345.00  |74345.00  |
|Portlanoregonusa                       |1    |150000.00 |150000.00 |150000.00 |
|Whatcom County Wa                      |1    |75000.00  |75000.00  |75000.00  |
|Drexel Hill Pa Usa         

In [8]:
spark.sql('''
    SELECT regexp_replace(initcap(location), '([^a-zA-Z ])', '') AS location, 
           count(1) AS count,
           bround(avg((annual_salary)), 2) AS avg_salary, 
           bround(min((annual_salary)), 2) AS min_salary,
           bround(max((annual_salary)), 2) AS max_salary
    FROM region_and_compensation
    GROUP BY 1
    HAVING count > 1
    ORDER BY 2 DESC
''').show(truncate=False)

+--------------------+-----+----------+----------+-----------+
|location            |count|avg_salary|min_salary|max_salary |
+--------------------+-----+----------+----------+-----------+
|Washington Dc       |471  |115146.13 |124.00    |750000.00  |
|New York Ny         |238  |147879.37 |6320.00   |3750000.00 |
|Boston Ma Usa       |219  |99853.39  |4500.00   |400000.00  |
|New York Ny Usa     |212  |131213.78 |10500.00  |790000.00  |
|Boston Ma           |207  |115992.59 |80.00     |2000007.00 |
|Chicago Il Usa      |185  |96279.79  |18000.00  |530000.00  |
|Chicago Il          |184  |106036.39 |65.00     |800000.00  |
|Seattle Wa Usa      |168  |115088.37 |140.00    |676195.00  |
|Washington Dc Usa   |166  |109046.07 |31200.00  |657758.00  |
|San Francisco Ca Usa|155  |146428.85 |165.00    |800000.00  |
|Seattle Wa          |154  |211115.90 |35000.00  |14000000.00|
|San Francisco       |151  |147458.91 |125.00    |500000.00  |
|San Francisco Ca    |150  |141510.11 |0.00      |45000

In [9]:
spark.sql('''
    SELECT city, 
           country, 
           count(1) AS count,
           bround(avg((annual_salary)), 2) AS avg_salary, 
           bround(min((annual_salary)), 2) AS min_salary,
           bround(max((annual_salary)), 2) AS max_salary
    FROM data_2
    WHERE city IS NOT NULL
    GROUP BY 1, 2
    ORDER BY 3 DESC
''').show(truncate=False)

+-------------+-------------------+-----+----------+----------+----------+
|city         |country            |count|avg_salary|min_salary|max_salary|
+-------------+-------------------+-----+----------+----------+----------+
|Chicago      |United States (US) |70   |105771.10 |11500.00  |780000.00 |
|Seattle      |United States (US) |69   |110575.86 |30720.00  |200000.00 |
|London       |United Kingdom (GB)|57   |64217.27  |22600.00  |150000.00 |
|Denver       |United States (US) |55   |94154.20  |110.00    |190000.00 |
|Atlanta      |United States (US) |51   |89457.45  |11000.00  |195000.00 |
|San Francisco|United States (US) |48   |146534.27 |145.00    |290000.00 |
|Austin       |United States (US) |47   |91658.83  |35000.00  |250000.00 |
|Toronto      |Canada (CA)        |47   |67058.40  |50.00     |150000.00 |
|Dallas       |United States (US) |46   |87591.28  |38000.00  |189000.00 |
|Los Angeles  |United States (US) |45   |111594.40 |42000.00  |250000.00 |
|Boston       |United Sta

In [10]:
spark.sql('''
    SELECT country, 
           count(1) AS count,
           bround(avg((annual_salary)), 2) AS avg_salary, 
           bround(min((annual_salary)), 2) AS min_salary,
           bround(max((annual_salary)), 2) AS max_salary
    FROM data_2
    GROUP BY 1
    HAVING count > 1
    ORDER BY max_salary  DESC
''').show(200, truncate=False)

+-----------------------+-----+------------+----------+---------------+
|country                |count|avg_salary  |min_salary|max_salary     |
+-----------------------+-----+------------+----------+---------------+
|Australia (AU)         |186  |662022979.57|90.00     |123123123123.00|
|Netherlands (NL)       |51   |169672.47   |43.00     |5523675.00     |
|Argentina (AR)         |3    |1158121.67  |15000.00  |3434365.00     |
|United States (US)     |2972 |85995.35    |0.00      |780000.00      |
|Norway (NO)            |29   |76927.93    |40120.00  |567000.00      |
|Denmark (DK)           |26   |60397.08    |5027.00   |205000.00      |
|Sweden (SE)            |40   |54347.70    |2840.00   |200000.00      |
|Switzerland (CH)       |35   |97603.70    |54000.00  |175000.00      |
|Germany (DE)           |119  |54917.07    |40.00     |175000.00      |
|Poland (PL)            |23   |35857.75    |3000.00   |156000.00      |
|United Kingdom (GB)    |239  |50647.95    |45.00     |155000.00

Due to the different approach for each `years_experiece` used by each dataset, an attempt of standardization needed to take place before unifying the three datasets.

In [11]:
#   - **Compensation by Experience**
#     - Visualize the relationship between cash compensation and other forms of compensation

# data 1
#  |-- experience_range: string (nullable = true)
#  |-- annual_salary: long (nullable = true)

# data 2
#  |-- annual_salary: long (nullable = true)
#  |-- annual_bonus: long (nullable = true)
#  |-- experience_range: string (nullable = true)

# data 3
#  |-- annual_salary: string (nullable = true)
#  |-- annual_bonus: string (nullable = true)
#  |-- years_experience: double (nullable = true)

spark.sql('''
    SELECT DISTINCT experience_range
    FROM data_1
''').show()

spark.sql('''
    SELECT DISTINCT experience_range
    FROM data_2
''').show()

spark.sql('''
    SELECT DISTINCT years_experience
    FROM data_3
    LIMIT 10
''').show()

+----------------+
|experience_range|
+----------------+
|41 years or more|
|    8 - 10 years|
|   11 - 20 years|
|   21 - 30 years|
|     2 - 4 years|
|  1 year or less|
|   31 - 40 years|
|       5-7 years|
+----------------+

+----------------+
|experience_range|
+----------------+
|             2-5|
|            null|
|             20+|
|             0-2|
|            5-10|
|           10-20|
+----------------+

+----------------+
|years_experience|
+----------------+
|             8.0|
|             0.0|
|             7.0|
|             3.5|
|             0.2|
|            1.25|
|             4.5|
|            9.75|
|             6.5|
|            null|
+----------------+



The range approach from datasets 1 and 2 was used, but due to their different ranges and notations, a new standard needed to be used. Unfortunately, due to some values being unable to fit in both ranges at the same time, some information were lost.

In [12]:
spark.sql('''
    SELECT DISTINCT regexp_replace(experience_range, '([^0-9-]+)', ''), experience_range
    FROM data_1
    ORDER BY 1
''').show()

+-------------------------------------------------+----------------+
|regexp_replace(experience_range, ([^0-9-]+), , 1)|experience_range|
+-------------------------------------------------+----------------+
|                                                1|  1 year or less|
|                                            11-20|   11 - 20 years|
|                                              2-4|     2 - 4 years|
|                                            21-30|   21 - 30 years|
|                                            31-40|   31 - 40 years|
|                                               41|41 years or more|
|                                              5-7|       5-7 years|
|                                             8-10|    8 - 10 years|
+-------------------------------------------------+----------------+



In [13]:
spark.sql('''
    SELECT regexp_replace(experience_range, '([^0-9-]+)', '') AS experience_range, 1 AS src
    FROM data_1
    UNION
    SELECT experience_range, 2 AS src
    FROM data_2
''').distinct().show()

+----------------+---+
|experience_range|src|
+----------------+---+
|           11-20|  1|
|           21-30|  1|
|            8-10|  1|
|             2-4|  1|
|               1|  1|
|           31-40|  1|
|             5-7|  1|
|              41|  1|
|             0-2|  2|
|           10-20|  2|
|            null|  2|
|            5-10|  2|
|             20+|  2|
|             2-5|  2|
+----------------+---+



In [14]:
spark.sql('''
    SELECT DISTINCT  experience_range, 
                     CASE experience_range 
                        WHEN '1' THEN '0-1'
                        WHEN '0-2' THEN '0-1'
                        WHEN '2-5' THEN '2-4'
                        WHEN '5-7' THEN '5-10'
                        WHEN '8-10' THEN '5-10'
                        WHEN '10-20' THEN '11-20'
                        WHEN '20+' THEN '21-30'
                        WHEN '41' THEN '41+'
                        ELSE experience_range
                        END as new_range
    FROM (
        SELECT regexp_replace(experience_range, '([^0-9-]+)', '') AS experience_range
        FROM data_1
        UNION
        SELECT experience_range
        FROM data_2
    )
    ORDER BY 2
''').show()

+----------------+---------+
|experience_range|new_range|
+----------------+---------+
|            null|     null|
|               1|      0-1|
|             0-2|      0-1|
|           10-20|    11-20|
|           11-20|    11-20|
|             2-4|      2-4|
|             2-5|      2-4|
|           21-30|    21-30|
|             20+|    21-30|
|           31-40|    31-40|
|              41|      41+|
|             5-7|     5-10|
|            8-10|     5-10|
|            5-10|     5-10|
+----------------+---------+



In [15]:
spark.sql('''
    SELECT DISTINCT CASE  
                        WHEN years_experience < 2 THEN '0-1'
                        WHEN years_experience < 5 THEN '2-4'
                        WHEN years_experience < 11 THEN '5-10'
                        WHEN years_experience < 21 THEN '11-20'
                        WHEN years_experience < 31 THEN '21-30'
                        WHEN years_experience < 41 THEN '31-40'
                        -- WHEN years_experience >= 41 THEN '41+'
                        ELSE years_experience
                    END as new_range
    FROM data_3
    ORDER BY 1
''').show()

+---------+
|new_range|
+---------+
|     null|
|      0-1|
|    11-20|
|      2-4|
|    21-30|
|    31-40|
|     45.0|
|     5-10|
|     55.0|
+---------+



In [16]:
spark.sql('''
    SELECT *
    FROM data_3
    WHERE years_experience >= 41
''').show()

+-------------------+-------------+------------+--------+----------------+--------+----------------+
|          timestamp|annual_salary|annual_bonus|currency|years_experience|location|       job_title|
+-------------------+-------------+------------+--------+----------------+--------+----------------+
|2016-03-21 13:41:50|         null|        null|        |            45.0|    null|           Troll|
|2016-03-21 13:42:22|       120.00|       12.00|        |            55.0|  4chinz|Professional hoe|
+-------------------+-------------+------------+--------+----------------+--------+----------------+



In [17]:
spark.sql('''
    SELECT annual_salary,
           annual_bonus,   
           CASE experience_range 
                WHEN '1' THEN '0-1'
                WHEN '0-2' THEN '0-1'
                WHEN '2-5' THEN '2-4'
                WHEN '5-7' THEN '5-10'
                WHEN '8-10' THEN '5-10'
                WHEN '10-20' THEN '11-20'
                WHEN '20+' THEN '21-30'
                WHEN '41' THEN '41+'
                ELSE experience_range
           END as experience_range
    FROM (
        SELECT annual_salary,
               NULL AS annual_bonus, 
               regexp_replace(experience_range, '([^0-9-]+)', '') AS experience_range
        FROM data_1
        UNION
        SELECT annual_salary, annual_bonus, experience_range
        FROM data_2
        WHERE experience_range IS NOT NULL
    )
    UNION
    SELECT annual_salary,
           annual_bonus,
           CASE  
                WHEN years_experience < 2 THEN '0-1'
                WHEN years_experience < 5 THEN '2-4'
                WHEN years_experience < 11 THEN '5-10'
                WHEN years_experience < 21 THEN '11-20'
                WHEN years_experience < 31 THEN '21-30'
                WHEN years_experience < 41 THEN '31-40'
           END as experience_range
    FROM data_3
    WHERE years_experience IS NOT NULL
    AND years_experience < 41
''').createOrReplaceTempView('experience_and_compensation')

There is an inconsistency on the final aggregation. The smaller amount of experience `0-1` was the one with a higher average salary. This is due to the fact that multiple currencies were used, and not standardized to USD. Specially the currencies that does not have "cents" or have a hugely lower value compared to USD, like PKR.

In [20]:
spark.sql('''
    SELECT experience_range, bround(avg(annual_salary), 2) AS avg_salary
    FROM experience_and_compensation
    GROUP BY 1
''').show(truncate=False)

+----------------+----------------+
|experience_range|avg_salary      |
+----------------+----------------+
|11-20           |4889249.452353  |
|41+             |1033418.320234  |
|21-30           |344990.531230   |
|2-4             |263089.627929   |
|5-10            |439116.574782   |
|0-1             |131160871.826281|
|31-40           |143902.259214   |
+----------------+----------------+



In [21]:
spark.sql('''
    SELECT *
    FROM data_3
    WHERE years_experience < 2
    ORDER BY annual_salary DESC
''').show(truncate=False)

+-------------------+-------------+------------+--------+----------------+-----------------+---------------------+
|timestamp          |annual_salary|annual_bonus|currency|years_experience|location         |job_title            |
+-------------------+-------------+------------+--------+----------------+-----------------+---------------------+
|2016-03-21 19:36:53|156000000.00 |0.00        |        |0.0             |Jakarta          |R&D Intern           |
|2020-02-06 07:26:12|36000000.00  |0.00        |        |1.0             |Cheonan          |Production plan      |
|2016-03-21 20:27:45|7500000.00   |0.00        |        |0.0             |Jakarta          |Application Developer|
|2016-03-31 00:32:16|700000.00    |null        |        |1.0             |Vancouver, B.C.  |Software Developer   |
|2016-03-21 23:29:27|600000.00    |null        |PKR     |1.0             |Karachi, Pakistan|Software Engineer    |
|2016-03-27 04:26:06|540000.00    |0.00        |INR     |0.6             |Bangal